In [2]:
import pyodbc
import mysql.connector
import dask.dataframe as dd
import pandas as pd
# from scripts.db_utils import create_ebay_connection
import os 
from dotenv import load_dotenv

load_dotenv()

def create_ebay_connection():
    """Create connection to the hosted SQL Server database for IP addresses."""
    try:
        connection = pyodbc.connect(
            f"DRIVER={{MySQL ODBC 9.1 Unicode Driver}};"
            f"SERVER={os.getenv('EBAY_DB_HOST')};"
            f"DATABASE={os.getenv('EBAY_DB_NAME')};"
            f"UID={os.getenv('EBAY_DB_USER')};"
            f"PWD={os.getenv('EBAY_DB_PASSWORD')}"
        )
        return connection
    except pyodbc.Error as e:
        print(f"Error connecting to hosted database: {e}")
        return None
# Database connection parameters
conn = create_ebay_connection()

query = "SELECT * FROM tbl_iptrace_user_activity where ams_trans_rsn_cd=0"




def fetch_data_in_chunks(conn, query, chunk_size=100000):
    """
    Fetch data from the database in chunks, then convert to Dask DataFrame.
    """
    offset = 0
    while True:
        # Modify the query to include pagination using LIMIT and OFFSET for chunks
        paginated_query = f"{query} LIMIT {chunk_size} OFFSET {offset}"
        
        # Fetch data in chunks using pandas (it handles column and row alignment internally)
        df_chunk = pd.read_sql_query(paginated_query, conn)
        
        if df_chunk.empty:
            break
        
        # Convert to Dask DataFrame
        dask_df = dd.from_pandas(df_chunk, npartitions=4)
        
        yield dask_df
        
        offset += chunk_size


dask_chunks = []

# Fetch chunks and store them in the list
for dask_df_chunk in fetch_data_in_chunks(conn, query, chunk_size=100000):
    dask_chunks.append(dask_df_chunk)

# Concatenate all chunks into a single Dask DataFrame
full_dask_df = dd.concat(dask_chunks)

# Now you can perform operations on full_dask_df
# result = full_dask_df.groupby('some_column').agg({'another_column': 'mean'})

# Compute the result
# computed_result = result.compute()

# Show the result
print(full_dask_df.head())


C:\Users\ramegupta\AppData\Local\Temp\ipykernel_12524\3663271736.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_chunk = pd.read_sql_query(paginated_query, conn)
C:\Users\ramegupta\AppData\Local\Temp\ipykernel_12524\3663271736.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_chunk = pd.read_sql_query(paginated_query, conn)
C:\Users\ramegupta\AppData\Local\Temp\ipykernel_12524\3663271736.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_chunk = pd.read_sql_query(paginated_query, conn)
C:\Users\ramegupta\

   id     CLNT_RMT_IP  ROI_CLICK_EVENT_TS  \
0   1  77.111.247.168 2020-08-02 18:37:01   
1   2   66.249.73.101 2020-08-21 03:36:38   
2   3  77.111.247.129 2020-08-21 03:36:38   
3   4   66.249.73.101 2020-08-21 03:36:33   
4   5  77.111.247.129 2020-08-21 03:36:33   

                                          BRWSR_NAME  ams_trans_rsn_cd  \
0  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...                 0   
1  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...                 0   
2  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...                 0   
3  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...                 0   
4  Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...                 0   

  ams_pblshr_id    buyer_id  is_vpn            click_id  
0    5574672411   valen_949       1  209245484389035777  
1    5574672411  morbanisaf       1  209453736892211586  
2    5574672411  morbanisaf       1  209453736892211586  
3    5574672411  morbanisaf       1  209453736892211

In [3]:
# Get the shape of the DataFrame (number of rows and columns)
num_rows, num_columns = full_dask_df.shape[0].compute(), full_dask_df.shape[1]

print(f"Rows: {num_rows}, Columns: {num_columns}")


Rows: 1254243, Columns: 9


In [1]:
# import dask.dataframe as dd
# import pandas as pd


# data_dict = {
#     "id": [1, 2, 3, 4, 5],
#     "CLNT_RMT_IP": ["77.111.247.168", "66.249.73.101", "77.111.247.129", "66.249.73.101", "77.111.247.129"],
#     "ROI_CLICK_EVENT_TS": ["2020-08-02 18:37:01", "2020-08-21 03:36:38", "2020-08-21 03:36:38", "2020-08-21 03:36:33", "2020-08-21 03:36:33"],
#     "BRWSR_NAME": [
#         "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36",
#         "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36",
#         "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36",
#         "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36",
#         "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36"
#     ],
#     "ams_trans_rsn_cd": [0, 0, 0, 0, 0],
#     "ams_pblshr_id": ["5574672411", "5574672411", "5574672411", "5574672411", "5574672411"],
#     "buyer_id": ["valen_949", "morbanisaf", "morbanisaf", "morbanisaf", "morbanisaf"],
#     "is_vpn": [1, 1, 1, 1, 1],
#     "click_id": ["209245484389035777", "209453736892211586", "209453736892211586", "209453736892211586", "209453736892211586"]
# }
# pandas_df = pd.DataFrame(data_dict)

# # Convert the Pandas DataFrame to a Dask DataFrame
# dask_df = dd.from_pandas(pandas_df, npartitions=1)

# # Display the Dask DataFrame
# print(dask_df)


Dask DataFrame Structure:
                  id CLNT_RMT_IP ROI_CLICK_EVENT_TS BRWSR_NAME ams_trans_rsn_cd ams_pblshr_id buyer_id is_vpn click_id
npartitions=1                                                                                                         
0              int64      string             string     string            int64        string   string  int64   string
4                ...         ...                ...        ...              ...           ...      ...    ...      ...
Dask Name: frompandas, 1 expression
Expr=df


In [4]:
full_dask_df.head()

,id,CLNT_RMT_IP,ROI_CLICK_EVENT_TS,BRWSR_NAME,ams_trans_rsn_cd,ams_pblshr_id,buyer_id,is_vpn,click_id
0,1,77.111.247.168,2020-08-02 18:37:01,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,0,5574672411,valen_949,1,209245484389035777
1,2,66.249.73.101,2020-08-21 03:36:38,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,0,5574672411,morbanisaf,1,209453736892211586
2,3,77.111.247.129,2020-08-21 03:36:38,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,0,5574672411,morbanisaf,1,209453736892211586
3,4,66.249.73.101,2020-08-21 03:36:33,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,0,5574672411,morbanisaf,1,209453736892211586
4,5,77.111.247.129,2020-08-21 03:36:33,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,0,5574672411,morbanisaf,1,209453736892211586


In [13]:
unique_ips_per_buyer = full_dask_df.groupby('buyer_id')['CLNT_RMT_IP'].unique().compute()

# Display the result
print(unique_ips_per_buyer)


buyer_id
*boilerman*          [35.178.179.218, 3.8.205.75, 66.249.73.128, 3....
*cycle*sam*                            [185.198.190.4, 178.238.10.115]
*hunter-gatherer*                                    [213.109.200.184]
*isidore*            [77.111.247.44, 77.111.247.137, 77.111.247.71,...
*katho*              [3.71.50.234, 3.75.132.246, 18.159.213.6, 18.1...
                                           ...                        
zohai-moham                                          [159.242.227.139]
zosztatni0813                                          [185.46.137.15]
zubintirani                                           [91.196.221.205]
zwockel1978                                           [172.226.108.57]
zzzmariezzz                                          [172.225.189.174]
Name: CLNT_RMT_IP, Length: 465400, dtype: object


In [6]:
# Total click count from each IP
total_clicks_per_ip = full_dask_df.groupby('CLNT_RMT_IP')['click_id'].count().compute()
print("Total clicks per IP:")
print(total_clicks_per_ip)


Total clicks per IP:
CLNT_RMT_IP
103.219.20.158    69
103.219.20.246    73
103.219.20.54     76
103.219.20.70     43
104.28.240.84     17
                  ..
66.249.64.2        5
157.230.101.95     1
172.226.49.46      1
185.93.180.110     1
192.142.227.22     1
Name: click_id, Length: 7915, dtype: int64


In [7]:
dask_df.columns

Index(['id', 'CLNT_RMT_IP', 'ROI_CLICK_EVENT_TS', 'BRWSR_NAME',
       'ams_trans_rsn_cd', 'ams_pblshr_id', 'buyer_id', 'is_vpn', 'click_id'],
      dtype='object')

In [5]:
# Find the total click count per user
clicks_per_user = dask_df.groupby('buyer_id').size().compute()

# Display the result
print(clicks_per_user)


buyer_id
morbanisaf    4
valen_949     1
dtype: int64


## Unique IPs associated with each user(for each publisher) and their click counts

In [ ]:
# Combine multiple operations before computing
result = dask_df.groupby('buyer_id')['CLNT_RMT_IP'].nunique().to_frame(name='unique_ips') \
                .merge(dask_df.groupby('buyer_id').size().to_frame(name='total_clicks'), 
                       left_index=True, right_index=True)

# Trigger computation once
result = result.compute()
print(result)


In [12]:
# Find the number of clicks per publisher, per user, and per IP
# clicks_per_publisher_user_ip = full_dask_df.groupby(['ams_pblshr_id', 'buyer_id', 'CLNT_RMT_IP']).size().compute()

# Display the result
clicks_per_publisher_user_ip_df = clicks_per_publisher_user_ip.reset_index(name='click_count')

# Export to CSV
clicks_per_publisher_user_ip_df.to_csv('../data/csv/clicks_per_publisher_user_ip.csv', index=False)



## How many users are associated with multiple publishers

In [9]:
# Group by buyer_id and count the number of unique publishers
users_multiple_publishers = full_dask_df.groupby('buyer_id')['ams_pblshr_id'].nunique()

# Filter users who have more than one unique publisher
users_multiple_publishers = users_multiple_publishers[users_multiple_publishers > 1]

# Compute the result
result = users_multiple_publishers.compute()
print(result)


buyer_id
beapos_90           2
frankopp            8
wizardccb           2
thailandthaiossi    3
oldbloke7511        6
                   ..
pinkruby666         2
pushfit0_1          2
stevebg53           2
dome1907            2
eisprinzhessin      2
Name: ams_pblshr_id, Length: 71896, dtype: int64


In [10]:
# Group by buyer_id and ams_pblshr_id to check the unique combinations
user_publisher_counts = full_dask_df.groupby('buyer_id')['ams_pblshr_id'].nunique()

# Get users with more than one unique publisher
users_with_multiple_publishers = user_publisher_counts[user_publisher_counts > 1]

# Compute the result
result = users_with_multiple_publishers.compute()
print(result)


buyer_id
segule_0             3
pueri58              2
moffettj             3
vogel1111            2
ttbebe_hr99lwfmu     2
                    ..
patrickgrumpy3       2
brie_9999            2
jatjat-1wfyltcqmt    2
jugendrausch         2
sm437946             2
Name: ams_pblshr_id, Length: 71896, dtype: int64


In [ ]:
# Group by buyer_id to get unique ams_pblshr_id per user
user_publishers = full_dask_df.groupby('buyer_id')['ams_pblshr_id'].unique()

# Filter users with more than one unique publisher
users_with_multiple_publishers = user_publishers[user_publishers.apply(len) > 1]

# Compute the result
result = users_with_multiple_publishers.compute()
print(result)


In [16]:
# unique_ips_per_buyer


# unique_ips_per_buyer_df = unique_ips_per_buyer.reset_index(name='unique_ip')
# unique_ips_per_buyer_df.head()
# Export to CSV
unique_ips_per_buyer_df.to_csv('../data/csv/unique_ips_per_buyer.csv', index=False)


In [21]:
# unique_ips_counts_per_buyer = full_dask_df.groupby('buyer_id')['CLNT_RMT_IP'].nunique().compute()
# unique_ips_counts_per_buyer_df = unique_ips_counts_per_buyer.reset_index(name='unique_ip')
# unique_ips_counts_per_buyer_df.head()
# Export to CSV
unique_ips_counts_per_buyer_df.to_csv('../data/csv/unique_ips_counts_per_buyer.csv', index=False)


In [25]:
value_counts = full_dask_df['CLNT_RMT_IP'].value_counts()

# Trigger the computation and get the result
# result = value_counts.compute()
value_counts_df = value_counts.reset_index()
value_counts_df.columns = ['7', 'count']

# Display the result
# print(result)


In [28]:
value_counts_df.to_csv('../data/csv/click_counts_from_unique_ips.csv', index=False, single_file=True)


['c:\\Users\\ramegupta\\OneDrive - eBay Inc\\Desktop\\Utkarsh\\vpn-annotator\\vpn_annotator\\data\\csv\\click_counts_from_unique_ips.csv']

## Same IP se multiple buyers

In [36]:
# Step 1: Group by IP and get the unique buyer IDs per IP
grouped_buyers = full_dask_df.groupby('CLNT_RMT_IP')['buyer_id'].unique()

# Step 2: Compute and convert to a Pandas DataFrame
grouped_buyers_df = grouped_buyers.compute().reset_index()

# Step 3: Filter out IPs with only 1 unique buyer (i.e., IPs with multiple buyers)
grouped_buyers_df = grouped_buyers_df[grouped_buyers_df['buyer_id'].apply(len) > 1]

# Step 4: Add a new column for the count of unique buyers
grouped_buyers_df['buyer_count'] = grouped_buyers_df['buyer_id'].apply(len)

# Now, you have a DataFrame with IPs, the list of buyers, and the count of unique buyers
print(grouped_buyers_df)

# If you want to return this as a Dask DataFrame, you can re-create it as Dask DataFrame
dask_filtered_df = dd.from_pandas(grouped_buyers_df, npartitions=4)  # adjust npartitions as needed


         CLNT_RMT_IP                                           buyer_id  \
0     103.219.20.158  [losbolles, 11oss, waterfoil36, 8135denis, jim...   
1     103.219.20.246  [theo-id, richardh4111, mmmfergus, gixerdave19...   
2      103.219.20.54  [mak911s, hj85wales, admiralcasual, krihod-60,...   
3      103.219.20.70  [olliejolly93, davesdomestics!, grelaine2012, ...   
4      104.28.240.84  [lilycom_0, isatelle0, aymacom0, sabbir1199, p...   
...              ...                                                ...   
7887   146.70.173.76                             [jotenn0, p.serna2012]   
7888    40.77.167.37                             [klitzekl, sweber1990]   
7890  172.225.187.70                               [schmidtc1, s7bpk40]   
7898  172.233.140.51                   [corn.international, sharo_9241]   
7902    66.249.66.76                        [marplatense1956, ndeals80]   

      buyer_count  
0              46  
1              47  
2              49  
3              37  

In [38]:
dask_filtered_df.head()
dask_filtered_df.to_csv('../data/csv/same_ip_multiple_users.csv', index=False, single_file=True)


['c:\\Users\\ramegupta\\OneDrive - eBay Inc\\Desktop\\Utkarsh\\vpn-annotator\\vpn_annotator\\data\\csv\\same_ip_multiple_users.csv']

## For an IP avg. count of clicks

In [45]:
full_dask_df.columns

Index(['id', 'CLNT_RMT_IP', 'ROI_CLICK_EVENT_TS', 'BRWSR_NAME',
       'ams_trans_rsn_cd', 'ams_pblshr_id', 'buyer_id', 'is_vpn', 'click_id'],
      dtype='object')

In [46]:

# Group by the 'CLNT_RMT_IP' and count the number of 'click_id' for each IP
ip_click_counts = full_dask_df.groupby('CLNT_RMT_IP')['click_id'].count()

# Now calculate the average of those counts
average_clicks_per_ip = ip_click_counts.mean().compute()

# Print the result
print("Average count of clicks per IP:", average_clicks_per_ip)


Average count of clicks per IP: 158.46405559065067


In [48]:
ip_buyer_counts = full_dask_df.groupby('CLNT_RMT_IP')['buyer_id'].nunique()

# Step 2: Filter for IPs that are associated with more than one buyer_id
multiple_user_ips = ip_buyer_counts[ip_buyer_counts > 1].index.compute()  # .compute() to get the actual list

# Step 3: Filter the original DataFrame to keep only rows where the IP is in the multiple_user_ips list
filtered_df = full_dask_df[full_dask_df['CLNT_RMT_IP'].isin(multiple_user_ips)]

# Step 4: Group by 'CLNT_RMT_IP' and count the number of clicks for each IP
ip_click_counts = filtered_df.groupby('CLNT_RMT_IP')['click_id'].count()

# Step 5: Calculate the average number of clicks per IP
average_clicks_per_ip = ip_click_counts.mean().compute()

# Print the result
print("Average count of clicks per IP (where IP is associated with multiple users):", average_clicks_per_ip)



Average count of clicks per IP (where IP is associated with multiple users): 171.79260695002077
